<a href="https://colab.research.google.com/github/prathyuma-08/VITISH-2023/blob/main/VITISH_2023_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing libraries

In [ ]:
import tensorflow
import cv2
import keras
from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img
import numpy as np
from keras.optimizers import Adam
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
import os
import json
import requests
from bs4 import BeautifulSoup
from PIL import Image
import re
import smtplib
from email.mime.text import MIMEText

# Web Extraction

In [ ]:
source_url = "https://sites.google.com/site/pornographydatabase/"
response = requests.get(source_url)
soup = BeautifulSoup(response.text, "html.parser")

image_urls = []
for img in soup.find_all("img"):
    image_url = img.get("src")
    if image_url:
        image_urls.append(image_url)

print(image_urls)

# Model Training - Classification



In [ ]:
!wget https://figshare.com/ndownloader/files/27771210

In [ ]:
!unzip /content/27771210

In [ ]:
# Define the image data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('/content/image dataset/train', target_size=(299, 299), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory('/content/image dataset/test', target_size=(299, 299), batch_size=32, class_mode='binary')

In [ ]:
# Load the pre-trained InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer with 1024 units and ReLU activation
x = Dense(1024, activation='relu')(x)
# x = Dense(512, activation='relu')(x)
#x = Dense(256, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# Add a final output layer with sigmoid activation for binary classification
predictions = Dense(1, activation='sigmoid')(x)

# Define the final model
model = Model(inputs=base_model.input, outputs=predictions)

87910968/87910968 [==============================] - 5s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=10, steps_per_epoch=10, validation_data=test_generator, validation_steps=10)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, steps=10)
print('Test accuracy:', test_acc)

In [ ]:
model.save('model_94.h5')

# Image prediction

In [ ]:
im = Image.open(requests.get('https://sites.google.com/site/pornographydatabase/_/rsrc/1468849941795/home/pornDB2.png', stream=True).raw)

In [ ]:
# Define a function to preprocess the input image
def preprocess_image(image):
    # Load the image
    #image = cv2.imread(image_path)
    # Resize the image to 224x224 pixels
    image = np.array(image)
    image = cv2.resize(image, (224, 224))
    # Convert the image to a 4D tensor
    image = np.expand_dims(image, axis=0)
    # Subtract the mean RGB values of the ImageNet dataset
    image = image - [123.68, 116.779, 103.939]
    return image

In [ ]:
img = preprocess_image(im)

# Use the ResNet50 model to make a prediction
resnet_prediction = model.predict(img)

# Print the predictions
print('ResNet50 prediction:', resnet_prediction)

1/1 [==============================] - 0s 34ms/step
ResNet50 prediction: [[5.834533e-37]]


# Video prediction

In [ ]:
# set video file path of input video with name and extension
vid = cv2.VideoCapture('/content/Trailer Launch - 5 Days to go _ PS 2 _ Mani Ratnam _ AR Rahman _ Subaskaran _ Lyca Productions.mp4')


if not os.path.exists('images'):
    os.makedirs('images')

#for frame identity
index = 0
while(True):
    # Extract images
    ret, frame = vid.read()
    # end of frames
    if not ret: 
        break
    # Saves images
    name = './images/frame' + str(index) + '.jpg'
    print ('Creating...' + name)
    cv2.imwrite(name, frame)

    # next frame
    index += 1

In [ ]:
dir_path = r'/content/images/'
res=[]

for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(dir_path+path)
print(res)

In [ ]:
for i in res:
  img = preprocess_image(i)
# Use the ResNet50 model to make a prediction
  resnet_prediction = model.predict(img)
# Print the predictions
  print('Prediction:', resnet_prediction)

## FINAL MODELS

### 1. DATA COLLECTION

In [ ]:
#1. Data Collection

# CREATE FOLDER
def folder_create(images):
	try:
		folder_name = input("Enter Folder Name:- ")
		# folder creation
		os.mkdir(folder_name)

	# if folder exists with that name, ask another name
	except:
		print("Folder Exist with that name!")
		folder_create()

	# image downloading start
	download_images(images, folder_name)


# DOWNLOAD ALL IMAGES FROM THAT URL
def download_images(images, folder_name):

	# initial count is zero
	count = 0

	# print total images found in URL
	print(f"Total {len(images)} Image Found!")

	# checking if images is not zero
	if len(images) != 0:
		for i, image in enumerate(images):
			# From image tag ,Fetch image Source URL

						# 1.data-srcset
						# 2.data-src
						# 3.data-fallback-src
						# 4.src

			# Here we will use exception handling

			# first we will search for "data-srcset" in img tag
			try:
				# In image tag ,searching for "data-srcset"
				image_link = image["data-srcset"]
				
			# then we will search for "data-src" in img
			# tag and so on..
			except:
				try:
					# In image tag ,searching for "data-src"
					image_link = image["data-src"]
				except:
					try:
						# In image tag ,searching for "data-fallback-src"
						image_link = image["data-fallback-src"]
					except:
						try:
							# In image tag ,searching for "src"
							image_link = image["src"]

						# if no Source URL found
						except:
							pass

			# After getting Image Source URL
			# We will try to get the content of image
			try:
				r = requests.get(image_link).content
				try:

					# possibility of decode
					r = str(r, 'utf-8')

				except UnicodeDecodeError:

					# After checking above condition, Image Download start
					with open(f"{folder_name}/images{i+1}.jpg", "wb+") as f:
						f.write(r)

					# counting number of image downloaded
					count += 1
			except:
				pass

		# There might be possible, that all
		# images not download
		# if all images download
		if count == len(images):
			print("All Images Downloaded!")
			
		# if all images not download
		else:
			print(f"Total {count} Images Downloaded Out of {len(images)}")

# MAIN FUNCTION START
def main(url):

	# content of URL
	r = requests.get(url)

	# Parse HTML Code
	soup = BeautifulSoup(r.text, 'html.parser')

	# find all images in URL
	images = soup.findAll('img')

	# Call folder create function
	folder_create(images)


# take url
url = input("Enter URL:- ")

# CALL MAIN FUNCTION
main(url)

### 2. UPSAMPLING

In [ ]:
#2. Upsampling

path_cs  = 'D:\CODING\Python programs\VITISH 23\content\celebrity-SFW'
path_gi  = 'D:\CODING\Python programs\VITISH 23\content\gore-images'
path_li  = 'D:\CODING\Python programs\VITISH 23\content\lingerie'
path_nu  = 'D:\CODING\Python programs\VITISH 23\content\nudity'
path_sa  = 'D:\CODING\Python programs\VITISH 23\content\sexual activity'

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

# Loop through images and generate augmented versions
def upsample(image_path):
    for filename in os.listdir(image_path):
        img = load_img(os.path.join(image_path, filename))  # Load image
        x = img_to_array(img)  # Convert to numpy array
        x = x.reshape((1,) + x.shape)  # Reshape to 4D array with single sample
        i = 0
        for batch in datagen.flow(x, batch_size=1, save_to_dir=image_path, save_prefix=filename.split('.')[0], save_format='jpg'):
            i += 1
            if i > 9:  # Generate 10 augmented versions of each image
                break

upsample(path_cs)
upsample(path_gi)
upsample(path_li)
upsample(path_nu)
upsample(path_sa)

### 3. WEB SCRAPING

In [ ]:
#3. Web scraping

def web_scrap(web_url):
  source_url = web_url
  response = requests.get(source_url)
  soup = BeautifulSoup(response.text, "html.parser")

  image_urls = []
  for img in soup.find_all("img", {"src": re.compile(".*\.(jpg|jpeg|png|gif)")}):
      image_url = img.get("src")
      if image_url:
          image_urls.append(image_url)

  print(image_urls)

  svg_urls = []
  for svg in soup.find_all("svg"):
      svg_url = svg.get("src")
      if svg_url:
          svg_urls.append(svg_url)

  print(svg_urls)

  gif_urls = []
  for gif in soup.find_all("img", {"src": re.compile(".*\.gif")}):
      gif_url = gif.get("src")
      if gif_url:
          gif_urls.append(gif_url)

  print(gif_urls)

  print(image_urls)
  return image_urls

### 4. IMAGE PREDICITON

In [ ]:
#4. Image prediction

def image_pred(web_url):
  result_url = []
  list_of_url = web_scrap(web_url)
  model = keras.models.load_model('model_94.h5')
  for f in list_of_url:
    im = Image.open(requests.get(f, stream=True).raw)
    img = preprocess_image(im)
    prediction = model.predict(img)
    print("Prediction: ",prediction)
    if prediction>=0.65: #threshold
      result_url.append(f)
  return result_url

### 5. VIDEO PREDICTION

In [ ]:
#5. Video prediction

def video_pred(vid_path):
  vid = cv2.VideoCapture(vid_path)
  if not os.path.exists('images'):
    os.makedirs('images')

#for frame identity
  index = 0
  while(True):
      # Extract images
      ret, frame = vid.read()
      # end of frames
      if not ret: 
          break
      # Saves images
      name = './images/frame' + str(index) + '.jpg'
      print ('Creating...' + name)
      cv2.imwrite(name, frame)

      # next frame
      index += 1

  dir_path = r'/content/images/'
  res=[]

  for path in os.listdir(dir_path):
      # check if current path is a file
      if os.path.isfile(os.path.join(dir_path, path)):
          res.append(dir_path+path)
  #print(res)

  for i in res:
    img = preprocess_image(i)
  # Use the ResNet50 model to make a prediction
    resnet_prediction = model.predict(img)
  # Print the predictions
    print('Prediction:', resnet_prediction)

### 6. REPORTING

In [ ]:
def send_mail(send_to_gmail,subject,message,url):
    gmail= "gmail id"
    password= "password"
    
    send_text = image_pred(url)
    # storing  the information to the server
    msg=MIMEText(send_text)
    msg['from']=gmail
    msg['to']=send_to_gmail
    msg['subject']= "Obscene Media Detected in the following URLs"

    # To connect the server
    server= smtplib.SMTP_SSL('smtp.gmail.com',465)
    #To login to the server
    server.login(gmail,password)
    # Sending the mail
    server.sendmail(gmail,send_to_gmail,(msg.as_string()))
    print('Mail sent')
    # Exiting the server
    server.quit()